In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from fake_useragent import UserAgent
import time
from random import randint
from time import sleep
from bs4 import BeautifulSoup
import string
import re
from decimal import *
from datetime import datetime

In [2]:
ua = UserAgent()
userAgent = ua.random
print(userAgent)

Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.116 Safari/537.36 Mozilla/5.0 (iPad; U; CPU OS 3_2 like Mac OS X; en-us) AppleWebKit/531.21.10 (KHTML, like Gecko) Version/4.0.4 Mobile/7B334b Safari/531.21.10


In [53]:
chrome_driver_path = "C:/Users/austi/Documents/Github_Repos/Imperial_Applied_Project/ChromeDriver/chromedriver.exe"
chrome_profile_path = r'user-data-dir=C:\Users\austi\AppData\Local\Google\Chrome\User Data\Default'
options = webdriver.ChromeOptions()
options.add_argument(chrome_profile_path)
#options.add_argument('headless')
options.add_argument(f'--user-agent={userAgent}')
options.add_argument("--start-maximized")
options.add_argument("--disable-popup-blocking")
options.add_argument("--disable-extensions")
options.add_argument("--no-sandbox")
options.add_argument("--disable-impl-side-painting")
options.add_argument("--disable-setuid-sandbox")
options.add_argument("--disable-seccomp-filter-sandbox")
options.add_argument("--disable-single-click-autofill")
options.add_experimental_option("excludeSwitches", ['enable-automation'])
options.add_argument("window-size=1920,1080")
#plug this into the chrome console: navigator.webdriver -> undefined

driver = webdriver.Chrome(executable_path=chrome_driver_path, options=options)

In [54]:
#Original pop-up Window
driver.get(r'C:\Users\austi\Documents\Github_Repos\Imperial_Applied_Project\Testing_HTML\pop_up_window.html')

In [9]:
#Zipcode Window
driver.get(r'C:\Users\austi\Documents\Github_Repos\Imperial_Applied_Project\Testing_HTML\48206.html')

In [44]:
#New Pop Up Window Test
driver.get(r'C:\Users\austi\Documents\Github_Repos\Imperial_Applied_Project\Testing_HTML\48206_pop_up_window.html')

In [17]:
#Vacant Lot Example
driver.get(r'C:\Users\austi\Documents\Github_Repos\Imperial_Applied_Project\Testing_HTML\Vacant_Lot_Example.html')

In [ ]:
#No Results Example
driver.get(r'C:\Users\austi\Documents\Github_Repos\Imperial_Applied_Project\Testing_HTML\48205_No_Results_Example.html')

In [57]:
print(driver.capabilities)
driver.get('https://intoli.com/blog/making-chrome-headless-undetectable/chrome-headless-test.html')
driver.save_screenshot("screenshot5.png")
driver.close()

{'acceptInsecureCerts': False, 'browserName': 'chrome', 'browserVersion': '75.0.3770.142', 'chrome': {'chromedriverVersion': '75.0.3770.90 (a6dcaf7e3ec6f70a194cc25e8149475c6590e025-refs/branch-heads/3770@{#1003})', 'userDataDir': 'C:\\Users\\austi\\AppData\\Local\\Google\\Chrome\\User Data\\Default'}, 'goog:chromeOptions': {'debuggerAddress': 'localhost:64672'}, 'networkConnectionEnabled': False, 'pageLoadStrategy': 'normal', 'platformName': 'windows nt', 'proxy': {}, 'setWindowRect': True, 'strictFileInteractability': False, 'timeouts': {'implicit': 0, 'pageLoad': 300000, 'script': 30000}, 'unhandledPromptBehavior': 'dismiss and notify'}


WebDriverException: Message: chrome not reachable
  (Session info: chrome=75.0.3770.142)


In [11]:
#listing_button = driver.find_element_by_id('listing-type').click()
#--disable-autofill-keyboard-accessory-view
#--disable-physical-keyboard-autocorrect
#--disable-single-click-autofill
#48126
#48201

In [62]:
#print(get_overviewStats(), '\n')
#print(get_textDescription(), '\n')
#print(get_factsAndFeatures(), '\n')
#print(get_homeDetails(), '\n')
#print(get_expandableFactsAndFeatures(), '\n')
#print(get_gpsCoordinates(), '\n')
print(get_homeValues(), '\n')

{'Zestimate': '181146', 'Upper_Sale_Estimate': '172000', 'Lower_Sale_Estimate': '190000'} 



In [8]:
expandable = driver.find_element_by_xpath("//*[contains(@class, 'ds-expandable-card-footer-text ds-text-button')]")
if expandable.text == 'See more facts and features':
    time.sleep(2)
    expandable.click()

In [61]:
def get_gpsCoordinates():
    data_dict = {}
    string_value = driver.find_element_by_xpath("//script[@type='application/ld+json']").get_attribute('innerHTML')
    gps_coords = re.findall('(\")(latitude)(\")\:([-+]?[0-9]\d*(\.\d+)?)\,(\")(longitude)(\")\:([-+]?[0-9]\d*(\.\d+)?)', string=string_value)
    latitude = gps_coords[0][1]
    latitude_num = gps_coords[0][3]
    longitude = gps_coords[0][6]
    longitude_num = gps_coords[0][8]
    data_dict.update({
        latitude: float(Decimal(latitude_num)),
        longitude: float(Decimal(longitude_num))
                     })
    
    return data_dict
    

def get_homeDetails():
    data_dict = {}
    details = driver.find_element_by_css_selector('div.ds-home-details-chip')
    
    price = details.find_element_by_xpath("//*[@class='ds-price']//span[@class='ds-value']").get_attribute('innerHTML')
    price = re.sub("[^\d\.]", "", str(price))
    
    address = driver.find_element_by_css_selector('h1.ds-address-container').text
    
    living_area = driver.find_element_by_css_selector('span.ds-bed-bath-living-area').text
    
    price_cut_string = driver.find_elements_by_css_selector('span.ds-price-change.ds-vertical-divider')
    if price_cut_string:
        price_cut_string = price_cut_string[0].text
        price_cut = re.findall('(\$)([0-9]\d*(\.\d+)?)', price_cut_string)[0][1]
        price_cut = float(Decimal(price_cut)*1000)
        price_cut_date = re.findall('(\d+\/\d+)', price_cut_string)[0]
    else:
        price_cut = None
        price_cut_date = None
    
    zestimate = driver.find_elements_by_css_selector('span.ds-estimate-value')
    if zestimate:
        zestimate = re.sub("[^\d\.]", "", str(zestimate[0].text))
    else:
        zestimate = None
    
    data_dict.update({
        'Price': price,
        'Address': address,
        'Zestimate': zestimate,
        'Price_Cut': price_cut,
        'Price_Cut_Date': price_cut_date,
        'Living_Area': living_area
    })
    
    return data_dict
    

def get_overviewStats():
    data_dict = {}
    stats = driver.find_element_by_css_selector('ul.ds-overview-stats')
    for stat in stats.find_elements_by_tag_name("li"):
        name = stat.find_element_by_css_selector('div.ds-body-small').get_attribute('innerHTML')
        name = dict_name_change(name=name)
        value = stat.find_element_by_css_selector('div.ds-overview-stat-value').get_attribute('innerHTML')
        data_dict.update({name: value})
        #sleep(randint(1,5))
    return data_dict

def get_textDescription():
    data_dict = {}
    overview = driver.find_element_by_css_selector('div.character-count-text-fold-container')
    text_data = overview.find_element_by_tag_name('div').get_attribute('innerHTML')
    data_dict.update({'Description': text_data})
    
    return data_dict

def get_factsAndFeatures():
    data_dict = {}
    for fact in driver.find_elements_by_css_selector('li.ds-home-fact-list-item'):
        label = fact.find_element_by_css_selector('span.ds-standard-label.ds-home-fact-label').get_attribute('innerHTML')
        label = dict_name_change(name=label)
        value = fact.find_element_by_css_selector('span.ds-body.ds-home-fact-value').get_attribute('innerHTML')
        data_dict.update({label:value})
    if not data_dict:
        #Run the backup script to handle non-standard data
        data_dict = get_backUpFactsAndFeatures()
    return data_dict

def get_backUpFactsAndFeatures():
    data_dict = {}
    backup_facts = driver.find_element_by_css_selector('ul.ds-home-fact-list.backup-facts')
    for table in backup_facts.find_elements_by_tag_name('table'):
        rows = table.find_elements_by_tag_name('tr')
        for row in rows:
            col = row.find_elements_by_tag_name('td')
            data_dict.update({dict_name_change(str(col[0].text)) : str(col[1].text)})
    return data_dict
    

def get_expandableFactsAndFeatures():
    data_dict = {}
    #expandable = driver.find_element_by_xpath("//*[contains(@class, 'ds-expandable-card-footer-text ds-text-button')]")
    #if expandable.text == 'See more facts and features':
        #time.sleep(1)
        #expandable.find_element_by_tag_name('button').click()
        #time.sleep(2)
    factsAndFeatures = driver.find_element_by_css_selector('div.ds-home-facts-and-features.reso-facts-features')#xpath("//div[@class='ds-expandable-card-collapsed']")
    driver.execute_script("arguments[0].scrollIntoView();", factsAndFeatures)
    time.sleep(5)
    button = factsAndFeatures.find_element_by_tag_name('button')
    time.sleep(5)
    button.click()
    time.sleep(5)
    
    for table in factsAndFeatures.find_elements_by_tag_name('table'):
        rows = table.find_elements_by_tag_name('tr')
        for row in rows:
            col = row.find_elements_by_tag_name('td')
            #print(str(row.text)) #+ ' ' + str(col.text))
            data_dict.update({dict_name_change(str(col[0].text)) : str(col[1].text)})
    return data_dict

def get_homeValues():
    data_dict = {}
    homeValues = driver.find_element_by_id('ds-home-values')
    driver.execute_script("arguments[0].scrollIntoView();", homeValues)
    time.sleep(3)
    zestimate_value = homeValues.find_element_by_xpath("//p[contains(@class, 'Text-sc')]").text
    zestimate_value = re.sub("[^\d\.]", "", zestimate_value)
    
    sale_range = homeValues.find_element_by_xpath("//span[contains(@class, 'Text-sc')]").text
    sale_range = re.sub("[^\d\.\-]", "", str(sale_range))
    range_numbers = re.findall("(\d+)(\-)(\d+)", sale_range)
    upper_range = range_numbers[0][0]
    lower_range = range_numbers[0][2]
    
    data_dict.update({
        'Zestimate': zestimate_value,
        'Upper_Sale_Estimate': upper_range,
        'Lower_Sale_Estimate': lower_range
    })
    
    return data_dict
    
def get_nearbySchoolRating():
    data_dict = {}
    schoolList = driver.find_element_by_css_selector('div.ds-nearby-schools-list')
    driver.execute_script("arguments[0].scrollIntoView();", schoolList)
    time.sleep(5)
    for school in schoolList.find_elements_by_css_selector('div.ds-school-row'):
        school
    
    
#####################################################################################################################
def dict_name_change(name):
    name = string.capwords(name)
    name = name.replace(' ', '_')
    ''.join(e for e in name if e.isalnum())
    return name

######################################################################################################################
def recaptcha_check(data):
    input("Solve the Recaptch and then Press Enter to continue...")
    restart_after_captcha(data = data)

def restart_after_captcha(data):
    input("Reset the Zip Code and Page Number; Press Enter to Continue...")
    #photo_cards = driver.find_element_by_css_selector('ul.photo-cards.photo-cards_wow')
    data_dict = data
    loop_cur_page(data= data_dict)

def update_house_dict(main_dict, zpid_value, house_data):
    return main_dict[zpid_value].update(house_data)

def loop_cur_page(data):
    photo_cards = driver.find_element_by_css_selector('ul.photo-cards.photo-cards_wow')
    
    data_dict = data
    for house in photo_cards.find_elements_by_css_selector("article"):
        #print(i.find_element_by_css_selector('a').get_attribute('aria-label')
        try:
            time.sleep(10)
            zpid = str(house.get_attribute("id"))
            if zpid in data_dict.keys():
                continue
            data_dict.update({zpid: {}})
            print(zpid)
            house.click()
            count+=1
            time.sleep(10)
            break_val = input("Press Enter to continue...")
            if break_val != '':
                break
            data_dict[zpid].update(get_overviewStats())
            time.sleep(2)
            data_dict[zpid].update(get_text_description())
            #data_dict = update_house_dict(main_dict=data_dict, zpid_value=zpid,)
            time.sleep(10)
        except NoSuchElementException:
            try:
                if driver.find_element_by_css_selector('div.captcha-container'):
                    recaptcha_check(data= data_dict)
                else:
                    print('No ZPID Exists Here')
            except NoSuchElementException:
                print('None')
                break
        #input("Press Enter to continue...")
        driver.find_element_by_css_selector('button.ds-close-lightbox-icon.hc-back-to-list').click()
    
    return data_dict

In [7]:
photo_cards = driver.find_element_by_css_selector('ul.photo-cards.photo-cards_wow')

In [8]:
count = 0
data_dict = {}
for i in photo_cards.find_elements_by_css_selector("article"):
    #print(i.find_element_by_css_selector('a').get_attribute('aria-label')
    try:
        time.sleep(10)
        zpid = str(i.get_attribute("id"))
        data_dict.update({zpid: {}})
        print(zpid)
        i.click()
        count+=1
        time.sleep(10)
        break_val = input("Press Enter to continue...")
        if break_val != '':
            break
        data_dict[zpid].update(get_overviewStats())
        time.sleep(10)
    except NoSuchElementException:
        try:
            if driver.find_element_by_css_selector('div.captcha-container'):
                input("Solve the Recaptch and then Press Enter to continue...")
                #restart the run from here
            else:
                print('No ZPID Exists Here')
        except NoSuchElementException:
            print('None')
    input("Press Enter to continue...")
    driver.find_element_by_css_selector('button.ds-close-lightbox-icon.hc-back-to-list').click()
#print(i.find_element_by_css_selector('h3.list-card-addr').get_attribute('innerHTML'))
print(count)

zpid_88703884
Press Enter to continue...
Press Enter to continue...
zpid_2083298038
Press Enter to continue...
Press Enter to continue...
zpid_2084165609
Press Enter to continue...d
3


In [9]:
data_dict

{'zpid_88703884': {'Time on Zillow': '82 days',
  'Views': '1,680',
  'Saves': '22'},
 'zpid_2083298038': {'Time on Zillow': '4 days', 'Views': '326', 'Saves': '8'},
 'zpid_2084165609': {}}

In [ ]:
window_before = driver.window_handles[0]
print(window_before)

for listing in photo_cards.find_elements_by_tag_name("li"):
    time.sleep(10)
    if listing.get_attribute('class') == 'nav-ad-empty':
        continue
    #print(driver.current_url)
    listing.click()
    #print(driver.current_url)
    time.sleep(10)
    #print(driver.window_handles)
    #driver.switch_to.alert()
    #window_after = driver.window_handles[1]
    #driver.switch_to.window(window_after)
    entered = input("Press Enter to continue...")
    if entered != '':
        break
    #driver.find_element_by_css_selector('ul.ds-overview-stats')
    data_dict.update(get_overviewStats())
    time.sleep(10)
    #stats = house.find_element_by_css_selector('ul.ds-overview-stats')
    
    #house.find_element_by_css_selector('div.character-count-text-fold-container').get_attribute('div')
    #sleep(randint(1,100))
    driver.find_element_by_css_selector('button.ds-close-lightbox-icon.hc-back-to-list').click()

In [46]:
<div class="character-count-text-fold-container"><div class="" style="white-space: pre-wrap; font-size: 15px; line-height: 1.5;">VERY NICE 4 BEDROOM HOME IN EXCELLENT NEIGHBORHOOD. HOME HAS AN OPEN LAYOUT AND WITH A LITTLE TLC IT COULD BE SPECTACULAR.</div></div>

In [48]:
driver.find_element_by_css_selector('button.ds-close-lightbox-icon.hc-back-to-list').click()
#<button class="ds-close-lightbox-icon hc-back-to-list" aria-label="close button"><svg width="24px" height="24px" xmlns="http://www.w3.org/2000/svg"><path d="M11.778 11.778L4 4l7.778 7.778L4 19.556l7.778-7.778zm0 0l7.778 7.778-7.778-7.778L19.556 4l-7.778 7.778z" stroke="#FFF" stroke-width="2" fill="none" fill-rule="evenodd" stroke-linecap="round" stroke-linejoin="round"></path></svg></button>

In [50]:
for listing in driver.find_element_by_css_selector('ul.photo-cards.photo-cards_wow'):
    house = driver.find_element_by_tag_name("li").text

TypeError: 'WebElement' object is not iterable

In [ ]:
search_box.send_keys(Keys.RETURN)

In [72]:
photo_cards = driver.find_element_by_css_selector('ul.photo-cards.photo-cards_wow')
house = photo_cards.find_element_by_tag_name("li").click()


In [7]:
driver.close()

WebDriverException: Message: chrome not reachable
  (Session info: chrome=75.0.3770.142)
